**Imports and Reading Dataset**

In [9]:
import datetime as dt
import pandas as pd
pd.set_option('display.max_columns', None)
pd.set_option('display.width', 500)
pd.set_option('display.float_format', lambda x: '%.2f' % x)

df = pd.read_excel("online_retail.xlsx", sheet_name="Year 2010-2011")

df.head()

,Invoice,StockCode,Description,Quantity,InvoiceDate,Price,Customer ID,Country
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,2010-12-01 08:26:00,2.55,17850.00,United Kingdom
1,536365,71053,WHITE METAL LANTERN,6,2010-12-01 08:26:00,3.39,17850.00,United Kingdom
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,2010-12-01 08:26:00,2.75,17850.00,United Kingdom
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,2010-12-01 08:26:00,3.39,17850.00,United Kingdom
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,2010-12-01 08:26:00,3.39,17850.00,United Kingdom


**General Information About Dataset**

In [10]:
def information(df):
    print("###############################    Shape  ##################################")
    print(df.shape)
    print("###############################    Types  ##################################")
    print(df.dtypes)
    print("###############################    Head   ##################################")
    print(df.head())
    print("###############################    Tail   ##################################")
    print(df.tail())
    print("###############################    NA     ##################################")
    print(df.isnull().sum())
    print("############################### Quantiles ##################################")
    print(df.describe([0, 0.05, 0.50, 0.95, 0.99, 1]).T)

information(df)

###############################    Shape  ##################################
(541910, 8)
###############################    Types  ##################################
Invoice                object
StockCode              object
Description            object
Quantity                int64
InvoiceDate    datetime64[ns]
Price                 float64
Customer ID           float64
Country                object
dtype: object
###############################    Head   ##################################
  Invoice StockCode                          Description  Quantity         InvoiceDate  Price  Customer ID         Country
0  536365    85123A   WHITE HANGING HEART T-LIGHT HOLDER         6 2010-12-01 08:26:00   2.55     17850.00  United Kingdom
1  536365     71053                  WHITE METAL LANTERN         6 2010-12-01 08:26:00   3.39     17850.00  United Kingdom
2  536365    84406B       CREAM CUPID HEARTS COAT HANGER         8 2010-12-01 08:26:00   2.75     17850.00  United Kingdom
3  536365  

**There is missing values. Description and Customer ID. We need to remove missing values**

In [11]:
df.dropna(inplace=True)
df.isnull().sum()

Invoice        0
StockCode      0
Description    0
Quantity       0
InvoiceDate    0
Price          0
Customer ID    0
Country        0
dtype: int64

**Unıque Product Number**

In [12]:
df["Description"].nunique()

3896

**Product Frequencies**

In [13]:
df["Description"].value_counts()

WHITE HANGING HEART T-LIGHT HOLDER    2070
REGENCY CAKESTAND 3 TIER              1905
JUMBO BAG RED RETROSPOT               1662
ASSORTED COLOUR BIRD ORNAMENT         1418
PARTY BUNTING                         1416
                                      ... 
ANTIQUE RASPBERRY FLOWER EARRINGS        1
WALL ART,ONLY ONE PERSON                 1
GOLD/AMBER DROP EARRINGS W LEAF          1
INCENSE BAZAAR PEACH                     1
PINK BAROQUE FLOCK CANDLE HOLDER         1
Name: Description, Length: 3896, dtype: int64

**The 'C' in the invoices shows the canceled transactions. Remove the canceled transactions from the dataset**

In [14]:
df[df["Invoice"].str.contains("C", na=False)].head()

,Invoice,StockCode,Description,Quantity,InvoiceDate,Price,Customer ID,Country
141,C536379,D,Discount,-1,2010-12-01 09:41:00,27.50,14527.00,United Kingdom
154,C536383,35004C,SET OF 3 COLOURED FLYING DUCKS,-1,2010-12-01 09:49:00,4.65,15311.00,United Kingdom
235,C536391,22556,PLASTERS IN TIN CIRCUS PARADE,-12,2010-12-01 10:24:00,1.65,17548.00,United Kingdom
236,C536391,21984,PACK OF 12 PINK PAISLEY TISSUES,-24,2010-12-01 10:24:00,0.29,17548.00,United Kingdom
237,C536391,21983,PACK OF 12 BLUE PAISLEY TISSUES,-24,2010-12-01 10:24:00,0.29,17548.00,United Kingdom


In [15]:
df = df[~df["Invoice"].str.contains("C", na=False)]

**Creating 'TotalCost'**

In [16]:
df["TotalCost"] = df["Quantity"] * df["Price"]
df.head()

,Invoice,StockCode,Description,Quantity,InvoiceDate,Price,Customer ID,Country,TotalCost
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,2010-12-01 08:26:00,2.55,17850.00,United Kingdom,15.30
1,536365,71053,WHITE METAL LANTERN,6,2010-12-01 08:26:00,3.39,17850.00,United Kingdom,20.34
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,2010-12-01 08:26:00,2.75,17850.00,United Kingdom,22.00
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,2010-12-01 08:26:00,3.39,17850.00,United Kingdom,20.34
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,2010-12-01 08:26:00,3.39,17850.00,United Kingdom,20.34


**Calculating RFM Metrics**

Recency: How recently a customer has made a purchase

Frequency: How often a customer makes a purchase 

Monetary: How much money a customer spends on purchases

In [28]:
analysis_date = df["InvoiceDate"].max()+ dt.timedelta(days=2) #To calculate recency

**Calculating Customer-based Recency, Frequency and Monetary Metrics**

In [36]:
RFM = df.groupby('Customer ID').agg({'InvoiceDate': lambda InvoiceDate: (today_date - InvoiceDate.max()).days,
                                     'Invoice': lambda Invoice: Invoice.nunique(),
                                     'TotalCost': lambda TotalCost: TotalCost.sum()})

RFM.head()

,InvoiceDate,Invoice,TotalCost
Customer ID,,,
12346.00,327,1,77183.60
12347.00,3,7,4310.00
12348.00,76,4,1797.24
12349.00,20,1,1757.55
12350.00,311,1,334.40


**Rename Columns**

In [37]:
RFM.columns = ['Recency', 'Frequency', 'Monetary']
RFM.describe().T

,count,mean,std,min,25%,50%,75%,max
Recency,4339.00,93.52,100.01,2.00,19.00,52.00,143.00,375.00
Frequency,4339.00,4.27,7.71,1.00,1.00,2.00,5.00,210.00
Monetary,4339.00,2053.80,8988.25,0.00,307.24,674.45,1661.64,280206.02


In [38]:
RFM = RFM[RFM["Monetary"] > 0]
RFM.describe().T # Monetary shoul be above 0

,count,mean,std,min,25%,50%,75%,max
Recency,4338.00,93.54,100.01,2.00,19.00,52.00,143.00,375.00
Frequency,4338.00,4.27,7.71,1.00,1.00,2.00,5.00,210.00
Monetary,4338.00,2054.27,8989.23,3.75,307.41,674.48,1661.74,280206.02


In [39]:
RFM["Recency_Score"] = pd.qcut(RFM['Recency'], 5, labels=[5, 4, 3, 2, 1])
RFM.head()

RFM["Frequency_Score"] = pd.qcut(RFM['Frequency'].rank(method="first"), 5, labels=[1, 2, 3, 4, 5])
RFM.head()

RFM["Monetary_Score"] = pd.qcut(RFM['Monetary'], 5, labels=[1, 2, 3, 4, 5])
RFM.head()

,Recency,Frequency,Monetary,Recency_Score,Frequency_Score,Monetary_Score
Customer ID,,,,,,
12346.00,327,1,77183.60,1,1,5
12347.00,3,7,4310.00,5,5,5
12348.00,76,4,1797.24,2,4,4
12349.00,20,1,1757.55,4,1,4
12350.00,311,1,334.40,1,1,2


**We need Recency and Frequency values to calculate RFM Metrics**

In [42]:
RFM["RF_SCORE"] = (RFM['Recency_Score'].astype(str) + RFM['Frequency_Score'].astype(str))
RFM.head()

,Recency,Frequency,Monetary,Recency_Score,Frequency_Score,Monetary_Score,RF_SCORE
Customer ID,,,,,,,
12346.00,327,1,77183.60,1,1,5,11
12347.00,3,7,4310.00,5,5,5,55
12348.00,76,4,1797.24,2,4,4,24
12349.00,20,1,1757.55,4,1,4,41
12350.00,311,1,334.40,1,1,2,11


**Segmentation**

In [43]:
seg_map = {
    r'[1-2][1-2]': 'hibernating',
    r'[1-2][3-4]': 'at_Risk',
    r'[1-2]5': 'cant_loose',
    r'3[1-2]': 'about_to_sleep',
    r'33': 'need_attention',
    r'[3-4][4-5]': 'loyal_customers',
    r'41': 'promising',
    r'51': 'new_customers',
    r'[4-5][2-3]': 'potential_loyalists',
    r'5[4-5]': 'champions'
}
RFM['Segment'] = RFM['RF_SCORE'].replace(seg_map, regex=True)
RFM.head()

,Recency,Frequency,Monetary,Recency_Score,Frequency_Score,Monetary_Score,RF_SCORE,Segment
Customer ID,,,,,,,,
12346.00,327,1,77183.60,1,1,5,11,hibernating
12347.00,3,7,4310.00,5,5,5,55,champions
12348.00,76,4,1797.24,2,4,4,24,at_Risk
12349.00,20,1,1757.55,4,1,4,41,promising
12350.00,311,1,334.40,1,1,2,11,hibernating


**Segment Details**

In [44]:
RFM[["Segment", "Recency", "Frequency", "Monetary"]].groupby("Segment").mean().sort_values("Monetary",ascending = False)

,Recency,Frequency,Monetary
Segment,,,
champions,6.88,12.42,6857.96
loyal_customers,34.47,6.46,2856.72
cant_loose,133.43,8.38,2796.16
at_Risk,156.06,2.87,1076.51
potential_loyalists,18.12,2.01,1034.91
need_attention,54.06,2.33,889.23
hibernating,218.90,1.10,487.71
about_to_sleep,54.50,1.16,461.06
new_customers,7.86,1.00,388.21
